In [ ]:
import dask.array as da
import xgboost as xgb
from dask_ml.model_selection import train_test_split
from dask_cuda import LocalCUDACluster
from dask.distributed import Client

In [ ]:
# Configurar Dask para usar GPUs
cluster = LocalCUDACluster()
client = Client(cluster)

In [ ]:
# Generar datos de ejemplo
X = da.random.random((10000, 20), chunks=(1000, 20))
y = da.random.random((10000,), chunks=(1000,))

In [ ]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Convertir los datos a DaskDMatrix para XGBoost
dtrain = xgb.dask.DaskDMatrix(client, X_train, y_train)
dtest = xgb.dask.DaskDMatrix(client, X_test, y_test)

In [ ]:
# Configuración del modelo
params = {
    'objective': 'reg:squarederror',
    'tree_method': 'hist',
    'device': 'cuda'
}

In [ ]:
# Entrenar el modelo
output = xgb.dask.train(client, params, dtrain, num_boost_round=100)

In [ ]:
# Predecir
bst = output['booster']
predictions = xgb.dask.predict(client, bst, dtest)

In [ ]:
# Evaluar
from sklearn.metrics import mean_squared_error
preds = predictions.compute()
labels = y_test.compute()
mse = mean_squared_error(labels, preds)
print(f'Mean Squared Error: {mse}')